## Computing PCA using RDDs

##  PCA

The vectors that we want to analyze have length, or dimension, of 365, corresponding to the number of 
days in a year.

We will perform [Principle component analysis (PCA)](https://en.wikipedia.org/wiki/Principal_component_analysis)
on these vectors. There are two steps to this process:

1) Computing the covariance matrix: this is a  simple computation. However, it takes a long time to compute and it benefits from using an RDD because it involves all of the input vectors.

2) Computing the eigenvector decomposition. this is a more complex computation, but it takes a fraction of a second because the size to the covariance matrix is $365 \times 365$, which is quite small. We do it on the head node usin `linalg`

### Computing the covariance matrix
Suppose that the data vectors are the column vectors denoted $x$ then the covariance matrix is defined to be
$$
E(x x^T)-E(x)E(x)^T
$$

Where $x x^T$ is the **outer product** of $x$ with itself.

If the data that we have is $x_1,x_2,x_n$ then  we estimate the covariance matrix:
$$
\hat{E}(x x^T)-\hat{E}(x)\hat{E}(x)^T
$$

the estimates we use are:
$$
\hat{E}(x x^T) = \frac{1}{n} \sum_{i=1}^n x_i x_i^T,\;\;\;\;\;
\hat{E}(x) = \frac{1}{n} \sum_{i=1}^n x_i
$$

## Computing the covariance matrix where the `nan`s are
### The effect of  `nan`s in arithmetic operations
* We use an RDD of numpy arrays, instead of Dataframes.
* Why? Because unlike dataframes, `numpy.nanmean` treats `nan` entries correctly.

### Calculating the mean of a vector with nan's
* We often get vectors $x$ in which some, but not all, of the entries are `nan`. 
* We want to compute the mean of the elements of $x$. 
* If we use `np.mean` we will get the result `nan`. 
* A useful alternative is to use `np.nanmean` which removes the `nan` elements and takes the mean of the rest.

In [1]:
%run '../spark_variables.ipynb'

In [2]:
import numpy as np
a=np.array([1,np.nan,2,np.nan,3,4,5])
print('a=',a)
print('np.mean(a)=',np.mean(a))
print('np.mean(np.nan_to_num(a))=',np.mean(np.nan_to_num(a))) # =(1+0+2+0+3+4+5)/7
print('np.nanmean(a)=',np.nanmean(a)) # =(1+2+3+4+5)/5

a= [ 1. nan  2. nan  3.  4.  5.]
np.mean(a)= nan
np.mean(np.nan_to_num(a))= 2.142857142857143
np.nanmean(a)= 3.0


### The outer poduct of a vector with `nan`s with itself

In [3]:
np.outer(a,a)

array([[ 1., nan,  2., nan,  3.,  4.,  5.],
       [nan, nan, nan, nan, nan, nan, nan],
       [ 2., nan,  4., nan,  6.,  8., 10.],
       [nan, nan, nan, nan, nan, nan, nan],
       [ 3., nan,  6., nan,  9., 12., 15.],
       [ 4., nan,  8., nan, 12., 16., 20.],
       [ 5., nan, 10., nan, 15., 20., 25.]])

### When should you not use `np.nanmean` ?
Using `n.nanmean` is equivalent to assuming that choice of which elements to remove is independent of the values of the elements. 
* Example of bad case: suppose the larger elements have a higher probability of being `nan`. In that case `np.nanmean` will under-estimate the mean

### Computing the covariance  when there are `nan`s
The covariance is a mean of outer products.

We calculate two matrices:
* $S$ - the sum of the matrices, whereh `nan`->0
* $N$ - the number of not-`nan` element for each matrix location.

We then calculate the mean as $S/N$ (division is done element-wise)

## Computing the mean together with the covariance
To compute the covariance matrix we need to compute both $\hat{E}(x x^T)$ and $\hat{E}(x)$. Using a simple trick, we can compute both at the same time.

Here is the trick: lets denote a $d$ dimensional **column vector** by $\vec{x} = (x_1,x_2,\ldots,x_d)$ (note that the subscript here is the index of the coordinate, not the index of the example in the training set as used above). 

The augmented vector $\vec{x}'$ is defined to be the $d+1$ dimensional vector $\vec{x}' = (1,x_1,x_2,\ldots,x_d)$.

The outer product of $\vec{x}'$ with itself is equal to 

$$ \vec{x}' {\vec{x}'}^T
= \left[\begin{array}{c|ccc}
    1 &  &{\vec{x}}^T &\\
    \hline \\
    \vec{x} & &\vec{x} {\vec{x}}^T \\ \\
    \end{array}
    \right]
$$

Where the lower left matrix is the original outer product $\vec{x} {\vec{x}}^T$ and the first row and the first column are $\vec{x}^T$ and $\vec{x}$ respectively.

Now suppose that we do the take the average of the outer product of the augmented vector and convince yourself that:
$$
\hat{E}(\vec{x}' {\vec{x}'}^T) = \frac{1}{n} \sum_{i=1}^n {\vec{x}'}_i {\vec{x}'}_i^T
= \left[\begin{array}{c|ccc}
    1 &  &\hat{E}(\vec{x})^T &\\
    \hline \\
    \hat{E}(\vec{x}) & &\hat{E}(\vec{x} {\vec{x}}^T) \\ \\
    \end{array}
    \right]
$$

So indeed, we have produced the outer product average together with (two copies of) the average $\hat{E}(\vec{x})$

In [4]:
# Set to True if running notebook on AWS/EMR
EMR=False 

In [5]:
if not EMR:
    import findspark
    findspark.init()
from pyspark import SparkContext,SparkConf

def create_sc(pyFiles):
    sc_conf = SparkConf()
    sc_conf.setAppName("Weather_PCA")
    sc_conf.set('spark.executor.memory', '3g')
    sc_conf.set('spark.executor.cores', '1')
    sc_conf.set('spark.cores.max', '4')
    sc_conf.set('spark.default.parallelism','10')
    sc_conf.set('spark.logConf', True)
    print(sc_conf.getAll())

    sc = SparkContext(conf=sc_conf,pyFiles=pyFiles)

    return sc 

sc = create_sc(pyFiles=['lib/numpy_pack.py','lib/spark_PCA.py','lib/computeStatistics.py'])

dict_items([('spark.app.name', 'Weather_PCA'), ('spark.executor.memory', '3g'), ('spark.executor.cores', '1'), ('spark.cores.max', '4'), ('spark.default.parallelism', '10'), ('spark.logConf', 'True')])


In [6]:
from pyspark.sql import *
sqlContext = SQLContext(sc)

import numpy as np
from lib.computeStatistics import *

### Climate data

The data we will use here comes from [NOAA](https://www.ncdc.noaa.gov/). Specifically, it was downloaded from This [FTP site](ftp://ftp.ncdc.noaa.gov/pub/data/ghcn/daily/).

There is a large variety of measurements from all over the world, from 1870 will 2012.
in the directory `../../Data/Weather` you will find the following useful files:

* data-source.txt: the source of the data
* ghcnd-readme.txt: A description of the content and format of the data
* ghcnd-stations.txt: A table describing the Meteorological stations.



### Data cleaning

* Most measurements exists only for a tiny fraction of the stations and years. We therefor restrict our use to the following measurements:
```python
['TMAX', 'SNOW', 'SNWD', 'TMIN', 'PRCP', 'TOBS']
```

* 8 We consider only measurement-years that have at most 50 `NaN` entries

* We consider only measurements in the continential USA

* We partition the stations into the states of the continental USA (plus a few stations from states in canada and mexico).

In [7]:
state='NY'
if not EMR:
    data_dir='../Data/Weather'
    tarname=state+'.tgz'
    parquet=state+'.parquet'
    
    %mkdir -p $data_dir
    !rm -rf $data_dir/$tarname

    command="curl https://mas-dse-open.s3.amazonaws.com/Weather/by_state_2/%s > %s/%s"%(tarname,data_dir,tarname)
    print(command)
    !$command
    !ls -lh $data_dir/$tarname

    cur_dir,=!pwd
    %cd $data_dir
    !tar -xzf $tarname
    !du ./$parquet
    %cd $cur_dir

curl https://mas-dse-open.s3.amazonaws.com/Weather/by_state_2/NY.tgz > ../Data/Weather/NY.tgz
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 63.2M  100 63.2M    0     0  29.3M      0  0:00:02  0:00:02 --:--:-- 29.3M
-rw-r--r--  1 brianroepke  staff    63M Sep 21 13:23 ../Data/Weather/NY.tgz
/Users/brianroepke/Projects/UCSanDiegoX/Big-Data/Data/Weather
155648	./NY.parquet
/Users/brianroepke/Projects/UCSanDiegoX/Big-Data/Section2-Weather-PCA_PART1


In [8]:
if EMR:  # not debugged, should use complete parquet and extract just the state of interest.
    data_dir='/mnt/workspace/Data'
    !hdfs dfs -mkdir /weather/
    !hdfs dfs -CopyFromLocal $data_dir/$parquet /weather/$parquet

    # When running on cluster
    #!mv ../../Data/Weather/NY.parquet /mnt/workspace/Data/NY.parquet

    !aws s3 cp --recursive --quiet /mnt/workspace/Data/NY.parquet s3://dse-weather/NY.parquet

    !aws s3 ls s3://dse-weather/

    local_path=data_dir+'/'+parquet
    hdfs_path='/weather/'+parquet
    local_path,hdfs_path

    !hdfs dfs -copyFromLocal $local_path $hdfs_path

    !hdfs dfs -du /weather/
    parquet_path=hdfs_path

In [9]:
parquet_path = data_dir+'/'+parquet
!du -sh $parquet_path

 76M	../Data/Weather/NY.parquet


In [10]:
%%time
df=sqlContext.read.parquet(parquet_path)
print(df.count())
df.show(5)

168398
+-----------+-----------+----+--------------------+-----------------+--------------+------------------+-----------------+-----+-----------------+
|    Station|Measurement|Year|              Values|       dist_coast|      latitude|         longitude|        elevation|state|             name|
+-----------+-----------+----+--------------------+-----------------+--------------+------------------+-----------------+-----+-----------------+
|USW00094704|   PRCP_s20|1945|[00 00 00 00 00 0...|361.8320007324219|42.57080078125|-77.71330261230469|208.8000030517578|   NY|DANSVILLE MUNI AP|
|USW00094704|   PRCP_s20|1946|[99 46 52 46 0B 4...|361.8320007324219|42.57080078125|-77.71330261230469|208.8000030517578|   NY|DANSVILLE MUNI AP|
|USW00094704|   PRCP_s20|1947|[79 4C 75 4C 8F 4...|361.8320007324219|42.57080078125|-77.71330261230469|208.8000030517578|   NY|DANSVILLE MUNI AP|
|USW00094704|   PRCP_s20|1948|[72 48 7A 48 85 4...|361.8320007324219|42.57080078125|-77.71330261230469|208.8000030517

In [11]:
sqlContext.registerDataFrameAsTable(df,'table')

In [12]:
Query="""
SELECT Measurement,count(Measurement) as count 
FROM table 
GROUP BY Measurement
ORDER BY count
"""
counts=sqlContext.sql(Query)
counts.show()

+-----------+-----+
|Measurement|count|
+-----------+-----+
|       TOBS|10956|
|   TOBS_s20|10956|
|   TMAX_s20|13437|
|       TMAX|13437|
|   TMIN_s20|13442|
|       TMIN|13442|
|       SNWD|14617|
|   SNWD_s20|14617|
|   SNOW_s20|15629|
|       SNOW|15629|
|   PRCP_s20|16118|
|       PRCP|16118|
+-----------+-----+



In [13]:
from time import time
t=time()

N=sc.defaultParallelism
print('Number of executors=',N)
print('took',time()-t,'seconds')

Number of executors= 10
took 0.0006940364837646484 seconds


In [14]:
!ls lib

MultiPlot.py            computeStatistics.py    row_parser.py
Reconstruction_plots.py decomposer.py           spark_PCA.py
YearPlotter.py          getFiles.py             spark_PCA_HW.py
__init__.py             import_modules.py       tmp
__pycache__             leaflet.py
binary_search.py        numpy_pack.py


In [15]:
# %load lib/spark_PCA.py
import numpy as np
from numpy import linalg as LA

def outerProduct(X):
    """Computer outer product and indicate which locations in matrix are undefined"""
    O=np.outer(X,X)
    N=1-np.isnan(O)
    return (O,N)

def sumWithNan(M1,M2):
    """Add two pairs of (matrix,count)"""
    (X1,N1)=M1
    (X2,N2)=M2
    N=N1+N2
    X=np.nansum(np.dstack((X1,X2)),axis=2)
    return (X,N)

### PA3: Replace the RHS of the expressions in this function (They need to depend on S and N.)
### You will complete this function for PA3: ComputingStatistics.ipynb
### For now, this function will return dummy data.
def HW_func(S,N):
    E=      np.ones([365]) # E is the sum of the vectors
    NE=     np.ones([365]) # NE is the number of not-nan antries for each coordinate of the vectors
    Mean=   np.ones([365]) # Mean is the Mean vector (ignoring nans)
    O=      np.ones([365,365]) # O is the sum of the outer products
    NO=     np.ones([365,365]) # NO is the number of non-nans in the outer product.
    return  E,NE,Mean,O,NO

In [16]:
def computeCov(RDDin):
    """computeCov recieves as input an RDD of np arrays, all of the same length, 
    and computes the covariance matrix for that set of vectors"""
    RDD=RDDin.map(lambda v:np.array(np.insert(v,0,1),dtype=np.float64)) # insert a 1 at the beginning of each vector so that the same 
                                           #calculation also yields the mean vector
    OuterRDD=RDD.map(outerProduct)   # separating the map and the reduce does not matter because of Spark uses lazy execution.
    (S,N)=OuterRDD.reduce(sumWithNan)

    E,NE,Mean,O,NO=HW_func(S,N)

    Cov=O/NO - np.outer(Mean,Mean)
    # Output also the diagnal which is the variance for each day
    Var=np.array([Cov[i,i] for i in range(Cov.shape[0])])
    return {'E':E,'NE':NE,'O':O,'NO':NO,'Cov':Cov,'Mean':Mean,'Var':Var}

In [17]:
if __name__=="__main__":
    # create synthetic data matrix with j rows and rank k
    
    V=2*(np.random.random([2,10])-0.5)
    data_list=[]
    for i in range(1000):
        f=2*(np.random.random(2)-0.5)
        data_list.append(np.dot(f,V))
    # compute covariance matrix
    RDD=sc.parallelize(data_list)
    OUT=computeCov(RDD)

    #find PCA decomposition
    eigval,eigvec=LA.eig(OUT['Cov'])
    print('eigval=',eigval)
    print('eigvec=',eigvec)

eigval= [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 

In [18]:
%%writefile lib/tmp
# %load lib/computeStatistics.py


from numpy import linalg as LA
import numpy as np

from numpy_pack import packArray,unpackArray
from spark_PCA import computeCov
from time import time

def computeStatistics(sqlContext,df):
    """Compute all of the statistics for a given dataframe
    Input: sqlContext: to perform SQL queries
            df: dataframe with the fields 
            Station(string), Measurement(string), Year(integer), Values (byteArray with 365 float16 numbers)
    returns: STAT, a dictionary of dictionaries. First key is measurement, 
             second keys described in computeStats.STAT_Descriptions
    """

    sqlContext.registerDataFrameAsTable(df,'weather')
    STAT={}  # dictionary storing the statistics for each measurement
    measurements=['TMAX', 'SNOW', 'SNWD', 'TMIN', 'PRCP', 'TOBS']
    
    for meas in measurements:
        t=time()
        Query="SELECT * FROM weather\n\tWHERE measurement = '%s'"%(meas)
        mdf = sqlContext.sql(Query)
        print(meas,': shape of mdf is ',mdf.count())

        data=mdf.rdd.map(lambda row: unpackArray(row['Values'],np.float16))

        #Compute basic statistics
        STAT[meas]=computeOverAllDist(data)   # Compute the statistics 

        # compute covariance matrix
        OUT=computeCov(data)

        #find PCA decomposition
        eigval,eigvec=LA.eig(OUT['Cov'])

        # collect all of the statistics in STAT[meas]
        STAT[meas]['eigval']=eigval
        STAT[meas]['eigvec']=eigvec
        STAT[meas].update(OUT)

        print('time for',meas,'is',time()-t)
    
    return STAT

# Compute the overall distribution of values and the distribution of the number of nan per year
def find_percentiles(SortedVals,percentile):
    L=int(len(SortedVals)/percentile)
    return SortedVals[L],SortedVals[-L]
  
def computeOverAllDist(rdd0):
    UnDef=np.array(rdd0.map(lambda row:sum(np.isnan(row))).sample(False,0.01).collect())
    flat=rdd0.flatMap(lambda v:list(v)).filter(lambda x: not np.isnan(x)).cache()
    count,S1,S2=flat.map(lambda x: np.float64([1,x,x**2]))\
                  .reduce(lambda x,y: x+y)
    mean=S1/count
    std=np.sqrt(S2/count-mean**2)
    Vals=flat.sample(False,0.0001).collect()
    SortedVals=np.array(sorted(Vals))
    low100,high100=find_percentiles(SortedVals,100)
    low1000,high1000=find_percentiles(SortedVals,1000)
    return {'UnDef':UnDef,\
          'mean':mean,\
          'std':std,\
          'SortedVals':SortedVals,\
          'low100':low100,\
          'high100':high100,\
          'low1000':low100,\
          'high1000':high1000
          }

# description of data returned by computeOverAllDist
STAT_Descriptions=[
('SortedVals', 'Sample of values', 'vector whose length varies between measurements'),
 ('UnDef', 'sample of number of undefs per row', 'vector whose length varies between measurements'),
 ('mean', 'mean value', ()),
 ('std', 'std', ()),
 ('low100', 'bottom 1%', ()),
 ('high100', 'top 1%', ()),
 ('low1000', 'bottom 0.1%', ()),
 ('high1000', 'top 0.1%', ()),
 ('E', 'Sum of values per day', (365,)),
 ('NE', 'count of values per day', (365,)),
 ('Mean', 'E/NE', (365,)),
 ('O', 'Sum of outer products', (365, 365)),
 ('NO', 'counts for outer products', (365, 365)),
 ('Cov', 'O/NO', (365, 365)),
 ('Var', 'The variance per day = diagonal of Cov', (365,)),
 ('eigval', 'PCA eigen-values', (365,)),
 ('eigvec', 'PCA eigen-vectors', (365, 365))
]




Overwriting lib/tmp


In [19]:
%%time 
### This is the main cell, where all of the statistics are computed. BE PATIENT, this will take considerable time.
STAT=computeStatistics(sqlContext,df)

TMAX : shape of mdf is  13437
time for TMAX is 37.56103706359863
SNOW : shape of mdf is  15629
time for SNOW is 43.665793895721436
SNWD : shape of mdf is  14617
time for SNWD is 38.86852478981018
TMIN : shape of mdf is  13442
time for TMIN is 36.39635396003723
PRCP : shape of mdf is  16118
time for PRCP is 45.522449254989624
TOBS : shape of mdf is  10956
time for TOBS is 31.723950147628784
CPU times: user 1.31 s, sys: 247 ms, total: 1.56 s
Wall time: 3min 53s


In [20]:
print("   Name  \t                 Description             \t  Size")
print("-"*80)
print('\n'.join(["%10s\t%40s\t%s"%(s[0],s[1],str(s[2])) for s in STAT_Descriptions]))

   Name  	                 Description             	  Size
--------------------------------------------------------------------------------
SortedVals	                        Sample of values	vector whose length varies between measurements
     UnDef	      sample of number of undefs per row	vector whose length varies between measurements
      mean	                              mean value	()
       std	                                     std	()
    low100	                               bottom 1%	()
   high100	                                  top 1%	()
   low1000	                             bottom 0.1%	()
  high1000	                                top 0.1%	()
         E	                   Sum of values per day	(365,)
        NE	                 count of values per day	(365,)
      Mean	                                    E/NE	(365,)
         O	                   Sum of outer products	(365, 365)
        NO	               counts for outer products	(365, 365)
       Cov	                

In [21]:
## Dump STAT and STST_Descriptions into a pickle file.
from pickle import dump

filename=data_dir+'/STAT_%s.pickle'%state
dump((STAT,STAT_Descriptions),open(filename,'wb'))
!ls -l $data_dir

total 179808
drwxr-xr-x  31 brianroepke  staff       992 Sep 21 13:23 NY.parquet
-rw-r--r--   1 brianroepke  staff  66288146 Sep 21 13:23 NY.tgz
-rw-r--r--   1 brianroepke  staff  25684694 Sep 21 13:28 STAT_NY.pickle


In [22]:
X=STAT['TMAX']['Var']
for key in STAT.keys():
    Y=STAT[key]['Var']
    print(key,sum(abs(X-Y)))

TMAX 0.0
SNOW 0.0
SNWD 0.0
TMIN 0.0
PRCP 0.0
TOBS 0.0


In [23]:
!ls -l ../Data/Weather/STAT*

-rw-r--r--  1 brianroepke  staff  25684694 Sep 21 13:28 ../Data/Weather/STAT_NY.pickle


In [24]:
!gzip -f -k ../Data/Weather/STAT*.pickle
!ls -l ../Data/Weather/STAT*

-rw-r--r--  1 brianroepke  staff  25684694 Sep 21 13:28 ../Data/Weather/STAT_NY.pickle
-rw-r--r--  1 brianroepke  staff     49284 Sep 21 13:28 ../Data/Weather/STAT_NY.pickle.gz


### Summary
* We discussed how to compute the covariance matrix and the expectation matrix when there are `nan` entries.
* The details are all in `computeStatistics`, which is defined in python files you can find in the directory `lib`